In [2]:
import os
import cv2
import shutil

def crop_and_save_images(image_directory, output_base_directory, label_directory):
    # 获取图像目录下所有图片文件
    image_files = [f for f in os.listdir(image_directory) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]

    for image_file in image_files:
        # 构建图像文件和标签文件的完整路径
        image_path = os.path.join(image_directory, image_file)
        label_path = os.path.join(label_directory, os.path.splitext(image_file)[0] + '.txt')

        # 检查标签文件是否存在
        if not os.path.exists(label_path):
            continue

        # 读取原始图像
        image = cv2.imread(image_path)

        # 创建输出目录
        output_directory = os.path.join(output_base_directory, os.path.splitext(image_file)[0])
        os.makedirs(output_directory, exist_ok=True)

        # 读取YOLO格式的检测结果
        with open(label_path, 'r') as file:
            lines = file.readlines()

        for line in lines:
            # 提取边界框信息
            class_id, x_center, y_center, width, height = map(float, line.strip().split())

            # 转换为绝对坐标
            img_height, img_width, _ = image.shape
            x, y, w, h = int((x_center - width / 2) * img_width), int((y_center - height / 2) * img_height), int(width * img_width), int(height * img_height)

            # 裁剪图像
            cropped_img = image[y:y+h, x:x+w]

            # 保存裁剪的图像
            output_file = os.path.join(output_directory, f"cropped_{class_id}_{x}_{y}_{w}_{h}.jpg")
            cv2.imwrite(output_file, cropped_img)

if __name__ == "__main__":
    # 图像文件目录
    image_directory = './Attachment_1/'
    # YOLO检测结果文件目录
    label_directory = './yolov5-master/runs/detect/exp2/labels/'
    # 输出基础目录（将会在此目录下创建与检测文件同名的目录）
    output_base_directory = './SegmentApple/'
    # 裁剪并保存图像
    crop_and_save_images(image_directory, output_base_directory, label_directory)